In [109]:
## attempt 4


import geopandas as gpd
from shapely.geometry import Point
from pyproj import Proj, transform


# Load the GeoJSON file containing multipolygon geometries
nyc_zones_in = gpd.read_file(r'C:\Users\TylerWoodstock\week14_15_project\notes\zoning\files\taxi_zones.json')

# Reproject the GeoDataFrame to EPSG:4326 (latitude-longitude)
source_crs = {'init': 'epsg:2263'}  # New York State Plane Coordinate System
target_crs = {'init': 'epsg:4326'}  # WGS84 (latitude/longitude)

nyc_zones = nyc_zones_in.to_crs(epsg=4326)


# Define the projection for LCC (replace these values with your specific projection parameters)

## NEW ATTEMPT (different and more parameters)
## THIS WORKS - CORRECT PROJECTION PARAMETERS
lcc_proj = Proj(
    proj='lcc',           # Projection type: Lambert Conformal Conic
    datum='WGS84',        # Datum: World Geodetic System 1984
    lat_1=40.66666666,    # First standard parallel (approximately 40°39'59.99996"N)
    lat_2=41.03333333,    # Second standard parallel (approximately 41°02'00.00002"N)
    lat_0=40.16666666,    # Latitude of origin (approximately 40°10'00.00002"N)
    lon_0=-74.00000000,   # Central meridian (approximately 74°00'00.00000"W)
    x_0=300000,           # False Easting (meters)
    y_0=0,                # False Northing (meters)
    units='us-ft',        # Units: US Survey Feet
    ellps='GRS80',        # Ellipsoid: Geodetic Reference System 1980
    no_defs=True          # Don't use additional projection parameters from the Proj definition file
)

# kept constant
wgs84_projection = Proj(proj='latlong', datum='WGS84', ellps='WGS84', no_defs=True)


#display(nyc_zones)
#print(nyc_zones.type)

# # Define a function to extract latitude and longitude from a geometry
def extract_lon_lat(row):
    # print(row)

    if row.type == 'MultiPolygon':

        lon_lat_list = []

        # Iterate over each polygon in the MultiPolygon
        for polygon in row.geoms:
            # Access exterior coordinates of the polygon
            x_coords, y_coords = polygon.exterior.coords.xy
            
            # Convert the exterior coordinates to latitude and longitude using the projection
            lon_lat_list.extend([(x, y) for x, y in zip(x_coords, y_coords)])

        return [transform(lcc_proj, wgs84_projection, x, y) for x, y in lon_lat_list]
    else:
        lon, lat = row.exterior.coords.xy
        return [transform(lcc_proj, wgs84_projection, x, y) for x, y in zip(lon, lat)]

# # Apply the function to the geometry column to get latitudes and longitudes
#nyc_zones['lon_lat'] = nyc_zones.apply(extract_lon_lat)

# test = nyc_zones.loc[1:4]['geometry'].apply(extract_lon_lat)
# display(test)

nyc_zones['lon_lat'] = nyc_zones['geometry'].apply(extract_lon_lat)
display(nyc_zones)







##### meta data #####
# # this successfully converts lcc to lat lon but ONLY for polygon objects, 
# # does not work for multipolygons (further iteration over the multipolygon needed)
# test_lon, test_lat = nyc_zones.loc[0]['geometry'].exterior.coords.xy
# lon_lat_coords = [transform(lcc_proj, wgs84_projection, x, y) for x, y in zip(test_lon, test_lat)]

## multipolygon solution
# lon_lat_list = []

# # Iterate over each polygon in the MultiPolygon
# for polygon in nyc_zones.loc[1]['geometry'].geoms:
#     # Access exterior coordinates of the polygon
#     x_coords, y_coords = polygon.exterior.coords.xy
    
#     # Convert the exterior coordinates to latitude and longitude using the projection
#     lon_lat_list.extend([(x, y) for x, y in zip(x_coords, y_coords)])

# # Convert LCC coordinates to latitude and longitude
# lon_lat_coords = [transform(lcc_proj, wgs84_projection, x, y) for x, y in lon_lat_list]





C:\Users\TylerWoodstock\AppData\Local\Temp\ipykernel_20424\3024487725.py:48: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if row.type == 'MultiPolygon':
C:\Users\TylerWoodstock\AppData\Local\Temp\ipykernel_20424\3024487725.py:63: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return [transform(lcc_proj, wgs84_projection, x, y) for x, y in zip(lon, lat)]
C:\Users\TylerWoodstock\AppData\Local\Temp\ipykernel_20424\3024487725.py:60: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return [transform(lcc_proj, wgs84_projection, x, y) for x, y in lon_lat_list]


,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,lon_lat
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.91835 192536.08570, 932771.39...","[(-74.18445299998285, 40.694995993327495), (-7..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1034866.45889 163365.73129, 10...","[(-73.81768666601685, 40.61493299132833), (-73..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1027837.26073 255725.65838, 1027727....","[(-73.84240629401471, 40.868474737325634), (-7..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((991277.85620 201171.19284, 991601.47...","[(-73.97464720800232, 40.71884234332722), (-73..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((936387.92190 143967.75556, 936046.56...","[(-74.172255076984, 40.56170483032897), (-74.1..."
...,...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((1029210.33870 265000.63630, 1029026....","[(-73.83737956162624, 40.893924734629806), (-7..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((1011545.88934 216046.87074, 1011466....","[(-73.90147053100911, 40.75963304332678), (-73..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((980288.14113 195963.02587, 980315.02...","[(-74.01428922899866, 40.70454906832737), (-74..."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((999884.64681 224506.38794, 999...","[(-73.94354420706748, 40.78288051748922), (-73..."


In [105]:
# Get subset of data

import pyarrow.parquet as pq
import pandas as pd


trips = pq.read_table(r'C:\Users\TylerWoodstock\week14_15_project\notes\zoning\files\yellow_tripdata_2009-01.parquet')
trips = trips.to_pandas()

trips_sub = trips.loc[0:100]

trips_sub_cols = trips_sub[['vendor_name', 'Trip_Distance', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat']]

display(trips_sub_cols[0:5])

,vendor_name,Trip_Distance,Start_Lon,Start_Lat,End_Lon,End_Lat
0,VTS,2.63,-73.991957,40.721567,-73.993803,40.695922
1,VTS,4.55,-73.982102,40.736290,-73.955850,40.768030
2,VTS,10.35,-74.002587,40.739748,-73.869983,40.770225
3,DDS,5.00,-73.974267,40.790955,-73.996558,40.731849
4,DDS,0.40,-74.001580,40.719382,-74.008378,40.720350


In [106]:
from shapely.geometry import Point, Polygon

# get start points for each trip
start_points = []
slat = list(trips_sub_cols['Start_Lat'])
slon = list(trips_sub_cols['Start_Lon'])

for lon, lat in zip(slon, slat):
    start_points.append(Point(lon, lat))

# get end points for each trip
end_points = []
elat = list(trips_sub_cols['End_Lat'])
elon = list(trips_sub_cols['End_Lon'])

for lon, lat in zip(elon, elat):
    end_points.append(Point(lon, lat))



#### testing
## empire state building test
# point = Point(-73.9857, 40.7484)
# print(point)

# # demo it works
# test = []
# for pnt in start_points[0:10]:
#     for idx, zone in nyc_zones.iterrows():
#         if pnt.within(Polygon(zone['lon_lat'])):
#             test.append(zone['zone'])
#             print({'zone': zone['zone'], 'LocationID': zone['LocationID']})
#####



# get pick up ids
pu_ids = []
pu_zones = []
for pnt in start_points:
    for idx, zone in nyc_zones.iterrows():
        if pnt.within(Polygon(zone['lon_lat'])):
            pu_ids.append(zone['LocationID'])
            pu_zones.append(zone['zone'])
            break

        if idx == len(nyc_zones) - 1:
            pu_ids.append(None)
            pu_zones.append(None)
        


# # get drop off ids
do_ids = []
do_zones = []
for pnt in end_points:
    for idx, zone in nyc_zones.iterrows():
        if pnt.within(Polygon(zone['lon_lat'])):
            do_ids.append(zone['LocationID'])
            do_zones.append(zone['zone'])
            break

        if idx == len(nyc_zones) - 1:
            do_ids.append(None)
            do_zones.append(None)   

# takes approx 1 min per 300 rows

print(len(pu_ids))
print(len(do_ids))
print(len(trips_sub_cols))


trips_sub_cols['pu_zone_id'] = pu_ids
trips_sub_cols['pu_zone'] = pu_zones
trips_sub_cols['do_zone_id'] = do_ids
trips_sub_cols['do_zone'] = do_zones

display(trips_sub_cols)

101
101
101


C:\Users\TylerWoodstock\AppData\Local\Temp\ipykernel_20424\42993489.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_sub_cols['pu_zone_id'] = pu_ids
C:\Users\TylerWoodstock\AppData\Local\Temp\ipykernel_20424\42993489.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_sub_cols['pu_zone'] = pu_zones
C:\Users\TylerWoodstock\AppData\Local\Temp\ipykernel_20424\42993489.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,vendor_name,Trip_Distance,Start_Lon,Start_Lat,End_Lon,End_Lat,pu_zone_id,pu_zone,do_zone_id,do_zone
0,VTS,2.63,-73.991957,40.721567,-73.993803,40.695922,148.0,Lower East Side,33.0,Brooklyn Heights
1,VTS,4.55,-73.982102,40.736290,-73.955850,40.768030,107.0,Gramercy,141.0,Lenox Hill West
2,VTS,10.35,-74.002587,40.739748,-73.869983,40.770225,249.0,West Village,70.0,East Elmhurst
3,DDS,5.00,-73.974267,40.790955,-73.996558,40.731849,238.0,Upper West Side North,113.0,Greenwich Village North
4,DDS,0.40,-74.001580,40.719382,-74.008378,40.720350,144.0,Little Italy/NoLiTa,231.0,TriBeCa/Civic Center
...,...,...,...,...,...,...,...,...,...,...
96,CMT,19.20,-74.006649,40.716653,-73.785956,40.638868,231.0,TriBeCa/Civic Center,NaN,None
97,CMT,1.80,-73.975951,40.740491,-73.989242,40.757063,137.0,Kips Bay,230.0,Times Sq/Theatre District
98,CMT,1.10,-73.972743,40.786563,-73.956258,40.779586,239.0,Upper West Side South,236.0,Upper East Side North
99,CMT,0.60,-73.959820,40.779339,-73.957995,40.774494,236.0,Upper East Side North,236.0,Upper East Side North


In [ ]:
## create a function version

from shapely.geometry import Point, Polygon

def 

# get start points for each trip
start_points = []
slat = list(trips_sub_cols['Start_Lat'])
slon = list(trips_sub_cols['Start_Lon'])

for lon, lat in zip(slon, slat):
    start_points.append(Point(lon, lat))

# get end points for each trip
end_points = []
elat = list(trips_sub_cols['End_Lat'])
elon = list(trips_sub_cols['End_Lon'])

for lon, lat in zip(elon, elat):
    end_points.append(Point(lon, lat))



# get pick up ids
pu_ids = []
pu_zones = []
for pnt in start_points:
    for idx, zone in nyc_zones.iterrows():
        if pnt.within(Polygon(zone['lon_lat'])):
            pu_ids.append(zone['LocationID'])
            pu_zones.append(zone['zone'])
            break

        if idx == len(nyc_zones) - 1:
            pu_ids.append(None)
            pu_zones.append(None)
        


# # get drop off ids
do_ids = []
do_zones = []
for pnt in end_points:
    for idx, zone in nyc_zones.iterrows():
        if pnt.within(Polygon(zone['lon_lat'])):
            do_ids.append(zone['LocationID'])
            do_zones.append(zone['zone'])
            break

        if idx == len(nyc_zones) - 1:
            do_ids.append(None)
            do_zones.append(None)   

# takes approx 1 min per 300 rows

print(len(pu_ids))
print(len(do_ids))
print(len(trips_sub_cols))


trips_sub_cols['pu_zone_id'] = pu_ids
trips_sub_cols['pu_zone'] = pu_zones
trips_sub_cols['do_zone_id'] = do_ids
trips_sub_cols['do_zone'] = do_zones

display(trips_sub_cols)